In [ ]:
pip install gensim

In [ ]:
pip install --upgrade gensim

In [ ]:
pip install sumy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21696 sha256=f5b920378164aa578dd2d9a370e83953b3e0059f4d198c51c2d1a3ae96070adf
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=08139f6516787f24f85c56d5db931acf558462643f277de9befd6b589826e829
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=1068

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import requests
from bs4 import BeautifulSoup
import gensim
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from sumy.parsers.html import HtmlParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer


In [ ]:
# Function to scrape the content of a web page (content is the eneregy certificate and not the report)
def scrape_webpage(url):
    response = requests.get(url)
    content = response.text
    return content

# Function to summarize the content using LexRank algorithm
def summarize_content(content, sentences_count=3):
    parser = HtmlParser.from_string(content, None, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, sentences_count)
    return " ".join(str(sentence) for sentence in summary)

# URL of the web page to scrape and summarize
url = "https://find-energy-certificate.service.gov.uk/energy-certificate/9671-3064-0910-0700-9495"  # Replace with the actual URL of the web page you want to scrape

# Scrape the web page content
content = scrape_webpage(url)

# Summarize the content
summary = summarize_content(content)

# Print the summary
print(summary)


Properties can be let if they have an energy rating from A+ to E. This property’s current energy rating is E. the efficiency of the property’s heating system power station efficiency for electricity the energy used to produce the fuel and deliver it to the property


In [ ]:


# Define a function for cleaning and preprocessing the text data
def preprocess_text(text):

    # Convert to lowercase
    summary = text.lower()

    # Remove non-alphanumeric characters
    summary = re.sub(r'[^a-zA-Z0-9\s]', '', summary)

    # Tokenize the text into words
    words = word_tokenize(summary)

    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

# Preprocess the text
cleaned_text = preprocess_text(summary)

# Train the Word2Vec model
model = gensim.models.Word2Vec([cleaned_text], min_count=1, vector_size=100)

# Vectorize the words
word_vectors = []
existing_words = set()

for sentence in cleaned_text:
    for word in sentence:
        if word in model.wv.key_to_index and word not in existing_words:
            word_vectors.append(model.wv.get_vector(word))

# Create a dataframe to store the word vectors
word_vectors_array = np.array(word_vectors)
reshaped_array = np.reshape(word_vectors_array, (1, -1))
df = pd.DataFrame(reshaped_array)
#df = pd.DataFrame(data=word_vectors, index=existing_words)

print(df)

       0         1         2         3         4         5         6     \
0 -0.008246  0.009299 -0.000196 -0.001968  0.004607 -0.004093  0.002744   

       7        8         9     ...      2790      2791      2792      2793  \
0  0.006943  0.00607 -0.007512  ... -0.007427 -0.001065 -0.000793 -0.002567   

       2794      2795      2796      2797      2798      2799  
0  0.009686 -0.000458  0.005875 -0.007446 -0.002506 -0.005546  

[1 rows x 2800 columns]


In [ ]:
url = "https://find-energy-certificate.service.gov.uk/energy-certificate/9243-1656-8157-5614-7032"

response = requests.get(url)
content = response.text

soup = BeautifulSoup(content, 'html.parser')
text = soup.get_text()

# Define a function for cleaning and preprocessing the text data
def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Convert to lowercase
    text = text.lower()

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Tokenize the text into words
    words = word_tokenize(text)

    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    cleaned_text = [word for word in words if not word.isdigit()]

    # Join the cleaned words back into a single string
    cleaned_text = " ".join(words)

    return cleaned_text




# Example Usage
# Assuming 'text_data' contains the extracted text from the web source
cleaned_text = preprocess_text(text)

print(cleaned_text)

#model = gensim.models.Word2Vec(cleaned_text, min_count=1, vector_size=100)



# Assuming 'text_data' contains the extracted text from the web source
#cleaned_sentences = sent_tokenize(cleaned_text)

#vecs = pd.DataFrame([model[word] for sentence in cleaned_sentences for word in sentence if word in model.vocab])

#word_vectors = vecs.values
#words = [word for sentence in cleaned_sentences for word in sentence if word in model.vocab]
#df = pd.DataFrame(cleaned_text)





energy performance certificate epc find energy certificate govuk cooky find energy certificate use essential cooky make service work wed also like use analytics cooky understand use service make improvement accept analytics cooky reject analytics cooky view cooky youve accepted analytics cooky change cookie setting time youve rejected analytics cooky change cookie setting time hide cookie message skip main content govuk find energy certificate energy performance certificate epc recommendation report report content energy rating epc recommendation property report detail assessor detail report property share report email copy link clipboard print su5355 vicar lane leeds ls1 6ba report number 92431656815756147032 valid 3 february 2032 energy rating epc property current energy rating information property energy performance see epc property recommendation make change improve property energy efficiency recommended improvement grouped estimated time would take change pay assessor may also mak

In [ ]:
url = "https://find-energy-certificate.service.gov.uk/energy-certificate/9243-1656-8157-5614-7032"

response = requests.get(url)
content = response.text

soup = BeautifulSoup(content, 'html.parser')
text = soup.get_text()

# Define a function for cleaning and preprocessing the text data
def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Convert to lowercase
    text = text.lower()

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Tokenize the text into words
    words = word_tokenize(text)

    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

# Preprocess the text
cleaned_text = preprocess_text(text)

# Train the Word2Vec model
model = gensim.models.Word2Vec([cleaned_text], min_count=1, vector_size=100)

# Vectorize the words
word_vectors = []
existing_words = set()

for sentence in cleaned_text:
    for word in sentence:
        if word in model.wv.key_to_index and word not in existing_words:
            word_vectors.append(model.wv.get_vector(word))

# Create a dataframe to store the word vectors
word_vectors_array = np.array(word_vectors)
reshaped_array = np.reshape(word_vectors_array, (1, -1))
df = pd.DataFrame(reshaped_array)
#df = pd.DataFrame(data=word_vectors, index=existing_words)

print(df)

       0         1         2         3         4         5         6     \
0  0.007838  0.009745  0.000386  0.001561  0.003197 -0.003717  0.007622   

       7         8         9     ...      9290    9291      9292      9293  \
0  0.004565  0.000164  0.005176  ... -0.005351 -0.0094  0.006982 -0.005469   

       9294      9295      9296     9297     9298      9299  
0 -0.009151  0.002698 -0.004932 -0.00741  0.00972 -0.001602  

[1 rows x 9300 columns]


In [ ]:
import os

directory =
file_list = os.listdir(directory)
response = requests.get(url)
content = response.text

soup = BeautifulSoup(content, 'html.parser')
text = soup.get_text()

# Define a function for cleaning and preprocessing the text data
def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Convert to lowercase
    text = text.lower()

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Tokenize the text into words
    words = word_tokenize(text)

    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

# Preprocess the text
cleaned_text = preprocess_text(text)

# Train the Word2Vec model
model = gensim.models.Word2Vec([cleaned_text], min_count=1, vector_size=100)

# Vectorize the words
word_vectors = []
existing_words = set()

for sentence in cleaned_text:
    for word in sentence:
        if word in model.wv.key_to_index and word not in existing_words:
            word_vectors.append(model.wv.get_vector(word))

# Create a dataframe to store the word vectors
word_vectors_array = np.array(word_vectors)
reshaped_array = np.reshape(word_vectors_array, (1, -1))
df = pd.DataFrame(reshaped_array)
#df = pd.DataFrame(data=word_vectors, index=existing_words)

print(df)